In [1]:
import networkx as nx

from python import aml, orc, pes, rdk, sql

data_dir = "/Users/tee/Documents/Research/QM Simulations/Rotavera/propyl:1-ethyl-2-methyl Oxirane /Potential Energy Surfaces/PES Playground/data/"
file_path = data_dir + "test.db"
connection = sql.connect(file_path)
sql.initialize_database(connection)

/Users/tee/Documents/Research/QM Simulations/Rotavera/propyl:1-ethyl-2-methyl Oxirane /Potential Energy Surfaces/PES Playground/.pixi/envs/default/lib/python3.13/site-packages/pandas/_libs/tslibs/__init__.py:40: InvalidTZPathWarning: Invalid paths specified in PYTHONTZPATH environment variable. Paths should be absolute but found the following relative paths:
    1-ethyl-2-methyl Oxirane /Potential Energy Surfaces/PES Playground/.pixi/envs/default/share/zoneinfo
    1-ethyl-2-methyl Oxirane /Potential Energy Surfaces/PES Playground/.pixi/envs/default/share/tzinfo
  from pandas._libs.tslibs.conversion import localize_pydatetime


In [ ]:
reactants = rdk.mol_from_smiles("[CH2]C")

# goat_temp = orc.Templates.GOAT.XTB
# goat_pars = orc.ORCA_Parameters(8, 1000, 2, "goat.inp", "guess.xyz")

# opt_temp = orc.Templates.Optimization.WB97X_3C
# opt_pars = orc.ORCA_Parameters(8, 1000, 2, "opt.inp", "goat.globalminimum.xyz")

# freq_temp = orc.Templates.Optimization_Frequency.M062x_def2_TZVPP
# freq_pars = orc.ORCA_Parameters(8, 1000, 2, "freq.inp", "opt.xyz")

# calc_temp = orc.Templates.Single_Point_Calculation.CCSDT_F12RI_cc_pVSZ_F12
# calc_pars = orc.ORCA_Parameters(8, 1000, 2, "calc.inp", "freq.xyz")

# stationary_templates = [goat_temp, opt_temp, freq_temp, calc_temp]
# stationary_parameters = [goat_pars, opt_pars, freq_pars, calc_pars]

goat_pars = orc.ORCA_Parameters(
    xyz_in="guess.xyz",
    name_out="goat",
    multiplicity=2,
    functional="XTB",
    method_keywords="GOAT",
)

submit_dict = {}
G = nx.Graph()
for reaction_smarts in [
    rdk.Reaction_Templates.PROTON_TRANSFER,
    rdk.Reaction_Templates.RING_OPENING,
]:
    product_sets = rdk.run_reaction(reactants, reaction_smarts, isomorphs=False)
    graph = aml.process_rdkit_reaction(reactants, product_sets)
    to_submit = sql.enumerated_graph_into_database(graph, file_path)

    for amchi, kind in to_submit.items():
        if kind == "stationary":
            goat_path = orc.orca_inputs(amchi, goat_pars, data_dir)
            G.add_node(goat_path)

        elif kind == "transition":
            print(amchi)
            continue

for n in G.nodes():
    print(n)


AMChI=1/C3H7O/c4-3-2-1-5-2/h2,4H,1,3H2/t2-/m0/s1/k5-2/f5-1/r0
AMChI=1/C3H7O/c4-3-2-1-5-2/h2,4H,1,3H2/t2-/m1/s1/k5-2/f5-1/r0
AMChI=1/C3H7O/c4-2-3-1-5-2/h2,4H,1,3H2/t2-/m0/s1/k5-2/f5-1/r0
AMChI=1/C3H7O/c4-2-3-1-5-2/h2,4H,1,3H2/t2-/m1/s1/k5-2/f5-1/r0
AMChI=1/C3H7O/c1-2-3-4-5-1/h1-3H2/k5-4/f5-1/r0


In [3]:
submit_dict

{'AMChI=1/C3H7O/c1-2-3-4/h4H,1-3H2': ['/Users/tee/Documents/Research/QM Simulations/Rotavera/propyl:1-ethyl-2-methyl Oxirane /Potential Energy Surfaces/PES Playground/data/AMChI=1/C3H7O/c1-2-3-4/h4H,1-3H2/goat.sh',
  '/Users/tee/Documents/Research/QM Simulations/Rotavera/propyl:1-ethyl-2-methyl Oxirane /Potential Energy Surfaces/PES Playground/data/AMChI=1/C3H7O/c1-2-3-4/h4H,1-3H2/opt.sh',
  '/Users/tee/Documents/Research/QM Simulations/Rotavera/propyl:1-ethyl-2-methyl Oxirane /Potential Energy Surfaces/PES Playground/data/AMChI=1/C3H7O/c1-2-3-4/h4H,1-3H2/freq.sh',
  '/Users/tee/Documents/Research/QM Simulations/Rotavera/propyl:1-ethyl-2-methyl Oxirane /Potential Energy Surfaces/PES Playground/data/AMChI=1/C3H7O/c1-2-3-4/h4H,1-3H2/calc.sh'],
 'AMChI=1/C3H7O/c1-2-3-4/h2,4H,3H2,1H3': ['/Users/tee/Documents/Research/QM Simulations/Rotavera/propyl:1-ethyl-2-methyl Oxirane /Potential Energy Surfaces/PES Playground/data/AMChI=1/C3H7O/c1-2-3-4/h2,4H,3H2,1H3/goat.sh',
  '/Users/tee/Documents/R

In [ ]:
import automol

graph = automol.smiles.graph("[CH2]C[C@@H]1[C@H](C)O1")
geom = automol.graph.geometry(graph)
automol.geom.display(geom, label=True)


# for stereo in automol.graph.expand_stereo(graph):
#     geom = automol.graph.geometry(stereo)
#     print(automol.graph.smiles(stereo))
#     automol.geom.display(geom, label=True)

In [ ]:
goat_pars = orc.ORCA_Parameters(8, 1000, 2, "test.inp", "test.xyz")
goat_temp = orc.Templates.GOAT.XTB

pes.orca_inputs(
    "AMChI=1/C5H9O/c1-3-5-4(2)6-5/h4-5H,1,3H2,2H3/t4-,5+/m0/s1",
    goat_temp,
    goat_pars,
    data_dir,
)
